In [13]:
# Standard Imports
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_bloch_multivector
import random
import math
import matplotlib.pyplot as plt
import numpy as np

# Preparation for encoding
KEY_LENGTH = 500
random.seed(0)  # Deterministic random sequence

# Generating a random string of bits
alice_bits = ''.join(str(random.randint(0, 1)) for _ in range(KEY_LENGTH))

print("The bits Alice is going to send are: " + alice_bits[:10] + "...")



The bits Alice is going to send are: 1101111110...


In [14]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis (Z or X) for each bit."""
    bases_string = ""
    for _ in range(num_of_bases):
        randBasis = random.randint(0, 1)
        bases_string += "Z" if randBasis == 0 else "X"
    return bases_string

# Generate Alice's random bases
alice_bases = generate_random_bases(KEY_LENGTH)
print("The bases Alice is going to encode them in are: " + alice_bases[:10] + "...")

def encode(bits, bases):
    """This function encodes each bit into a qubit using the specified basis."""
    encoded_qubits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == "0" and basis == "Z":
            pass  # Do nothing
        elif bit == "1" and basis == "Z":
            qc.x(0)
        elif bit == "0" and basis == "X":
            qc.h(0)
        elif bit == "1" and basis == "X":
            qc.x(0)
            qc.h(0)
        encoded_qubits.append(qc)
    return encoded_qubits


The bases Alice is going to encode them in are: ZXXZZZZXZX...


In [15]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# Print circuits for the first 5 qubits
for i in range(5):
    print(encoded_qubits[i])
    
# Print a message indicating there are more circuits
print("etc.")


     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐┌───┐
  q: ┤ X ├┤ H ├
     └───┘└───┘
c: 1/══════════
               
     ┌───┐
  q: ┤ H ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
etc.


In [35]:
# Imports
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# First, check your qiskit version
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

# Import simulator from available modules
try:
    # Try different import paths based on your Qiskit version
    try:
        from qiskit import Aer
        simulator = Aer.get_backend('qasm_simulator')
        statevector_sim = Aer.get_backend('statevector_simulator')
        print("Using Aer simulator")
    except ImportError:
        try:
            from qiskit import BasicAer
            simulator = BasicAer.get_backend('qasm_simulator')
            statevector_sim = BasicAer.get_backend('statevector_simulator')
            print("Using BasicAer simulator")
        except ImportError:
            from qiskit.providers.basicaer import QasmSimulatorPy
            simulator = QasmSimulatorPy()
            from qiskit.providers.basicaer import StatevectorSimulatorPy
            statevector_sim = StatevectorSimulatorPy()
            print("Using direct simulator classes")
except Exception as e:
    print(f"Error importing simulators: {e}")
    print("Trying to continue with alternative imports...")

# Try to import execute
try:
    from qiskit import execute
    print("Using qiskit.execute")
except ImportError:
    try:
        from qiskit.execute_function import execute
        print("Using qiskit.execute_function.execute")
    except ImportError:
        # Define a simple execute function using run method
        def execute(circuit, backend, **kwargs):
            job = backend.run(circuit, **kwargs)
            return job
        print("Using custom execute function")

from qiskit.visualization import plot_bloch_multivector
import random
import math
import matplotlib.pyplot as plt
import numpy as np

# Optional: Enable inline plots in Jupyter
# %matplotlib inline

# Preparation for encoding
KEY_LENGTH = 500
random.seed(0)

# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1)
    alice_bits += str(randBit)
print("The bits Alice is going to send are: " + alice_bits[:10] + "...")

# Function to generate random bases
def generate_random_bases(num_of_bases):
    bases_string = ""
    for i in range(num_of_bases):
        randBasis = random.randint(0, 1)  # Flip Coin
        if randBasis == 0:
            bases_string += "Z"
        else:
            bases_string += "X"
    return bases_string

# Alice chooses a random basis for each bit
alice_bases = generate_random_bases(KEY_LENGTH)
print("The bases Alice is going to encode them in are: " + alice_bases[:10] + "...")

# Encoding the bits based on the chosen bases
def encode(bits, bases):
    encoded_qubits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)  # Create a quantum circuit for each qubit
        if bit == "0" and basis == "Z":
            encoded_qubits.append(qc)  # Do not apply any gates
        elif bit == "1" and basis == "Z":
            qc.x(0)  # Apply X Gate
            encoded_qubits.append(qc)
        elif bit == "0" and basis == "X":
            qc.h(0)  # Apply H Gate
            encoded_qubits.append(qc)
        elif bit == "1" and basis == "X":
            qc.x(0)  # Apply X Gate
            qc.h(0)  # Apply H Gate
            encoded_qubits.append(qc)
    return encoded_qubits

# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# Print circuits for the first 5 qubits
for i in range(5):
    print(encoded_qubits[i])

# Quantum Channel (sending the qubits)
QUANTUM_CHANNEL = encoded_qubits
bob_bases = generate_random_bases(KEY_LENGTH)  # Bob randomly chooses a basis for each bit
print("The bases Bob is going to decode them in are: " + bob_bases[:10] + "...")

# Measuring the qubits based on Bob's chosen bases
def measure(qubits, bases):
    bits = ""  # The results of measurements
    for qubit, basis in zip(qubits, bases):
        if basis == "Z":
            qubit.measure(0, 0)
        elif basis == "X":
            qubit.h(0)
            qubit.measure(0, 0)
        # Execute on the simulator
        try:
            result = execute(qubit, backend=simulator, shots=1).result()
            counts = result.get_counts()
            measured_bit = max(counts, key=counts.get)  # Max doesn't matter for simulator since there is only one shot
        except Exception as e:
            print(f"Error during measurement: {e}")
            measured_bit = "0"  # Default if there's an error
        bits += measured_bit
    return bits

# Bob's received bits
qubits_received = QUANTUM_CHANNEL  # Receive qubits from quantum channel
try:
    bob_bits = measure(qubits_received, bob_bases)
    print("The first few bits Bob received are: " + bob_bits[:10] + "...")
except Exception as e:
    print(f"Error during measurement: {e}")
    # Create random bits as fallback
    bob_bits = ''.join(random.choice(['0', '1']) for _ in range(KEY_LENGTH))
    print("Using randomly generated bits for Bob due to simulator error")

# Compare bases and sift the key
def sift_key(alice_bases, bob_bases, alice_bits, bob_bits):
    sifted_key_alice = ""
    sifted_key_bob = ""
    matching_indices = []
    
    for i in range(len(alice_bases)):
        if alice_bases[i] == bob_bases[i]:
            matching_indices.append(i)
            sifted_key_alice += alice_bits[i]
            sifted_key_bob += bob_bits[i]
    
    return sifted_key_alice, sifted_key_bob, matching_indices

# Sift the key based on matching bases
sifted_key_alice, sifted_key_bob, matching_indices = sift_key(alice_bases, bob_bases, alice_bits, bob_bits)

print(f"\nNumber of qubits sent: {KEY_LENGTH}")
print(f"Number of matching bases: {len(matching_indices)}")
print(f"Alice's sifted key: {sifted_key_alice[:20]}...")
print(f"Bob's sifted key:   {sifted_key_bob[:20]}...")

# Check for errors (which would indicate eavesdropping)
error_count = 0
for a, b in zip(sifted_key_alice, sifted_key_bob):
    if a != b:
        error_count += 1

error_rate = error_count / len(sifted_key_alice) if sifted_key_alice else 0
print(f"\nError rate: {error_rate:.4f}")

# Final key after error checking
if error_rate < 0.25:  # Typically, error rates > 0.25 indicate possible eavesdropping
    print("No significant eavesdropping detected.")
    final_key = sifted_key_alice
    print(f"Final shared key: {final_key[:20]}...")
else:
    print("Possible eavesdropping detected! Aborting key exchange.")
    final_key = ""

# Try to visualize a qubit state (with error handling)
try:
    if len(encoded_qubits) > 0:
        # Create a sample qubit state for visualization
        test_qc = QuantumCircuit(1)
        
        # Apply the same transformations as one of the encoded qubits
        sample_index = 0
        bit = alice_bits[sample_index]
        basis = alice_bases[sample_index]
        
        if bit == "1":
            test_qc.x(0)
        if basis == "X":
            test_qc.h(0)
        
        # Execute and get statevector
        result = execute(test_qc, statevector_sim).result()
        statevector = result.get_statevector()
        
        # Plot Bloch vector
        figure = plot_bloch_multivector(statevector)
        plt.title(f"Qubit State (Bit: {bit}, Basis: {basis})")
        plt.show()
except Exception as e:
    print(f"Error during visualization: {e}")
    print("Skipping visualization")

Qiskit version: 1.3.3
Error importing simulators: No module named 'qiskit.providers.basicaer'
Trying to continue with alternative imports...
Using custom execute function
The bits Alice is going to send are: 1101111110...
The bases Alice is going to encode them in are: ZXXZZZZXZX...
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐┌───┐
  q: ┤ X ├┤ H ├
     └───┘└───┘
c: 1/══════════
               
     ┌───┐
  q: ┤ H ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
The bases Bob is going to decode them in are: XZZXZZZXXX...
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error du

In [36]:
# After Bob receives and measures the qubits

# Alice and Bob communicate over a classical channel to compare bases
CLASSICAL_CHANNEL = alice_bases  # Alice tells Bob which bases she used

# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i] == bob_bases[i]]
print("The indices of the first 10 bases they share in common are: " + str(common_bases[:10]))

# Convert string of bits to a list if necessary
if isinstance(bob_bits, str):
    bob_bits_list = list(bob_bits)
else:
    bob_bits_list = bob_bits

# Bob keeps only the bits where they used the same basis
bob_bits_filtered = [bob_bits_list[index] for index in common_bases]

# Bob tells Alice which bases they shared in common
CLASSICAL_CHANNEL = common_bases  

# Alice keeps only the bits they shared in common
if isinstance(alice_bits, str):
    alice_bits_list = list(alice_bits)
else:
    alice_bits_list = alice_bits

alice_bits_filtered = [alice_bits_list[index] for index in common_bases]

# Alice tells Bob the first 100 bits she has left (or all if less than 100)
comparison_length = min(100, len(alice_bits_filtered))
CLASSICAL_CHANNEL = alice_bits_filtered[:comparison_length]

# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits_filtered[:comparison_length]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

# Calculate how many bits they have for their shared key
key_length = len(common_bases)
print(f"Alice and Bob now share a key of {key_length} bits")

# Print a sample of their shared key (first 20 bits)
alice_key = ''.join(alice_bits_filtered)
bob_key = ''.join(bob_bits_filtered)
print(f"Alice's key (first 20 bits): {alice_key[:20]}")
print(f"Bob's key (first 20 bits):   {bob_key[:20]}")

# Calculate the error rate (percentage of bits that don't match)
error_count = sum(1 for a, b in zip(alice_bits_filtered, bob_bits_filtered) if a != b)
error_rate = error_count / len(alice_bits_filtered) if alice_bits_filtered else 0
print(f"Error rate: {error_rate:.4%}")

# Check if the error rate is acceptable (below a threshold)
error_threshold = 0.15  # 15% is a common threshold for QKD
if error_rate < error_threshold:
    print("No significant eavesdropping detected - key exchange successful!")
else:
    print("Warning: High error rate detected! Possible eavesdropping or channel noise.")
    print("Key exchange aborted for security reasons.")

The indices of the first 10 bases they share in common are: [4, 5, 6, 7, 9, 10, 11, 19, 22, 24]
Uh oh, at least one of the bits is different.
Alice and Bob now share a key of 223 bits
Alice's key (first 20 bits): 11110011110001011000
Bob's key (first 20 bits):   00000000000000000000
Error rate: 47.5336%
Key exchange aborted for security reasons.


In [37]:
# After verifying the first 100 bits match, discard them for security
# (since they were shared over the classical channel)

# Convert to lists if they're strings
if isinstance(alice_bits_filtered, str):
    alice_bits_filtered = list(alice_bits_filtered)
if isinstance(bob_bits_filtered, str):
    bob_bits_filtered = list(bob_bits_filtered)

# Ensure we have at least 100 bits before discarding
if len(alice_bits_filtered) > 100:
    alice_bits = alice_bits_filtered[100:]  # Alice discards the first 100 bits
    bob_bits = bob_bits_filtered[100:]      # Bob discards the first 100 bits
else:
    print("Warning: Less than 100 bits available. Using all remaining bits for the key.")
    alice_bits = alice_bits_filtered
    bob_bits = bob_bits_filtered

# Generate the final key from Alice's bits
# (Bob's bits should be identical at this point)
key = ""
for bit in alice_bits:  # Or bob_bits, since both should be the same
    key += str(bit)  # Ensure bit is treated as string when concatenating

print("Shhhhh, the key is:")
print(str(key))
print("Don't tell anyone!")
print("\nThe key is " + str(len(key)) + " bits long.")

# Optional: Display key security level based on length
if len(key) >= 256:
    print("This key provides very strong security (equivalent to AES-256 or better).")
elif len(key) >= 128:
    print("This key provides strong security (equivalent to AES-128).")
elif len(key) >= 64:
    print("This key provides moderate security.")
else:
    print("Warning: This key is relatively short for cryptographic purposes.")
    print("For production use, more qubits should be exchanged.")

Shhhhh, the key is:
010101101001011110110100011011001100000111010101000101011101001000110101100111101001101010000111000111000011001110100100000
Don't tell anyone!

The key is 123 bits long.
This key provides moderate security.


In [38]:
# Imports
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import qiskit
print(f"Qiskit version: {qiskit.__version__}")

# Import simulator from available modules
try:
    # Try different import paths based on your Qiskit version
    try:
        from qiskit import Aer
        simulator = Aer.get_backend('qasm_simulator')
        statevector_sim = Aer.get_backend('statevector_simulator')
        print("Using Aer simulator")
    except ImportError:
        try:
            from qiskit import BasicAer
            simulator = BasicAer.get_backend('qasm_simulator')
            statevector_sim = BasicAer.get_backend('statevector_simulator')
            print("Using BasicAer simulator")
        except ImportError:
            from qiskit.providers.basicaer import QasmSimulatorPy
            simulator = QasmSimulatorPy()
            from qiskit.providers.basicaer import StatevectorSimulatorPy
            statevector_sim = StatevectorSimulatorPy()
            print("Using direct simulator classes")
except Exception as e:
    print(f"Error importing simulators: {e}")
    print("Trying to continue with alternative imports...")

# Try to import execute
try:
    from qiskit import execute
    print("Using qiskit.execute")
except ImportError:
    try:
        from qiskit.execute_function import execute
        print("Using qiskit.execute_function.execute")
    except ImportError:
        # Define a simple execute function using run method
        def execute(circuit, backend, **kwargs):
            job = backend.run(circuit, **kwargs)
            return job
        print("Using custom execute function")

from qiskit.visualization import plot_bloch_multivector
import random
import math
import matplotlib.pyplot as plt
import numpy as np

# Optional: Enable inline plots in Jupyter
# %matplotlib inline

# Function to generate random bases
def generate_random_bases(num_of_bases):
    bases_string = ""
    for i in range(num_of_bases):
        randBasis = random.randint(0, 1)  # Flip Coin
        if randBasis == 0:
            bases_string += "Z"
        else:
            bases_string += "X"
    return bases_string

# Encoding the bits based on the chosen bases
def encode(bits, bases):
    encoded_qubits = []
    # Ensure bits is a string
    if not isinstance(bits, str):
        bits = ''.join(str(bit) for bit in bits)
        
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)  # Create a quantum circuit for each qubit
        if bit == "0" and basis == "Z":
            encoded_qubits.append(qc)  # Do not apply any gates
        elif bit == "1" and basis == "Z":
            qc.x(0)  # Apply X Gate
            encoded_qubits.append(qc)
        elif bit == "0" and basis == "X":
            qc.h(0)  # Apply H Gate
            encoded_qubits.append(qc)
        elif bit == "1" and basis == "X":
            qc.x(0)  # Apply X Gate
            qc.h(0)  # Apply H Gate
            encoded_qubits.append(qc)
    return encoded_qubits

# Measuring the qubits based on chosen bases
def measure(qubits, bases):
    bits = ""  # The results of measurements
    for qubit, basis in zip(qubits, bases):
        if basis == "Z":
            qubit.measure(0, 0)
        elif basis == "X":
            qubit.h(0)
            qubit.measure(0, 0)
        # Execute on the simulator
        try:
            result = execute(qubit, backend=simulator, shots=1).result()
            counts = result.get_counts()
            measured_bit = max(counts, key=counts.get)
        except Exception as e:
            print(f"Error during measurement: {e}")
            measured_bit = "0"  # Default if there's an error
        bits += measured_bit
    return bits

# Set parameters for the protocol
KEY_LENGTH = 500
random.seed(0)  # For reproducibility

# Generating a random string of bits for Alice
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1)  # Flip Coin
    alice_bits += str(randBit)  # Add randomly chosen bit to the bit string.

print(f"Alice's original bits (first 10): {alice_bits[:10]}...")

# Alice randomly chooses a basis for each bit
alice_bases = generate_random_bases(KEY_LENGTH)
print(f"Alice's bases (first 10): {alice_bases[:10]}...")

# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)
print(f"Alice encoded {len(encoded_qubits)} qubits")

# Qubits are sent through the quantum channel
QUANTUM_CHANNEL = encoded_qubits

# Eve intercepts the qubits
qubits_intercepted = QUANTUM_CHANNEL  # Intercept qubits
eve_bases = generate_random_bases(KEY_LENGTH)  # Eve guesses bases randomly
print(f"Eve's bases (first 10): {eve_bases[:10]}...")

# Eve measures the intercepted qubits
eve_bits = measure(qubits_intercepted, eve_bases)
print(f"Eve's measured bits (first 10): {eve_bits[:10]}...")

# Eve encodes her own qubits using her measurements and sends them to Bob
QUANTUM_CHANNEL = encode(eve_bits, eve_bases)
print(f"Eve forwarded {len(QUANTUM_CHANNEL)} qubits")

# Bob receives the qubits (which are now Eve's qubits, not Alice's original ones)
bob_bases = generate_random_bases(KEY_LENGTH)  # Bob randomly chooses bases
print(f"Bob's bases (first 10): {bob_bases[:10]}...")

# Bob measures the received qubits
qubits_received = QUANTUM_CHANNEL
bob_bits = measure(qubits_received, bob_bases)
print(f"Bob's measured bits (first 10): {bob_bits[:10]}...")

# Classical communication begins
# Alice and Bob publicly compare their bases
CLASSICAL_CHANNEL = alice_bases  # Alice tells Bob which bases she used

# They identify where they used the same basis
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i] == bob_bases[i]]
print(f"Indices of first 10 matching bases: {common_bases[:10]}")
print(f"Total matching bases: {len(common_bases)}")

# Bob filters his bits to keep only those measured with matching bases
bob_bits_filtered = [bob_bits[index] for index in common_bases]

# Bob tells Alice which bases they shared in common
CLASSICAL_CHANNEL = common_bases

# Alice filters her bits to keep only those encoded with matching bases
alice_bits_filtered = [alice_bits[index] for index in common_bases]

# Verification step: Alice shares a subset of her bits to check for eavesdropping
sample_size = min(100, len(alice_bits_filtered))
CLASSICAL_CHANNEL = alice_bits_filtered[:sample_size]  # Alice tells Bob the first bits

# Bob checks if they match his bits
matching = sum(1 for a, b in zip(CLASSICAL_CHANNEL, bob_bits_filtered[:sample_size]) if a == b)
error_rate = 1 - (matching / sample_size) if sample_size > 0 else 0

print(f"\nVerification result: {matching} matching bits out of {sample_size}")
print(f"Error rate: {error_rate:.2%}")

# Bob tells Alice if the bits match
if matching == sample_size:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print(f"Uh oh, {sample_size - matching} of the bits are different.")
    
# Check if error rate exceeds threshold
error_threshold = 0.15  # 15% is a common threshold
if error_rate > error_threshold:
    print("\nWARNING: High error rate detected! Possible eavesdropping!")
    print("Key exchange should be aborted for security.")
else:
    print("\nError rate within acceptable limits.")
    
    # If they decide to proceed, discard the verification bits and use the rest as the key
    if len(alice_bits_filtered) > sample_size:
        final_alice_key = ''.join(alice_bits_filtered[sample_size:])
        final_bob_key = ''.join(bob_bits_filtered[sample_size:])
        
        print(f"\nFinal key length: {len(final_alice_key)} bits")
        print(f"Alice's key (first 20): {final_alice_key[:20]}...")
        print(f"Bob's key (first 20): {final_bob_key[:20]}...")
        
        # Calculate actual differences in the keys (which shouldn't be known in real scenario)
        key_differences = sum(1 for a, b in zip(final_alice_key, final_bob_key) if a != b)
        final_error_rate = key_differences / len(final_alice_key) if final_alice_key else 0
        print(f"Actual differences in final keys: {key_differences} bits ({final_error_rate:.2%})")
    else:
        print("Not enough bits remaining after verification.")

Qiskit version: 1.3.3
Error importing simulators: No module named 'qiskit.providers.basicaer'
Trying to continue with alternative imports...
Using custom execute function
Alice's original bits (first 10): 1101111110...
Alice's bases (first 10): ZXXZZZZXZX...
Alice encoded 500 qubits
Eve's bases (first 10): XZZXZZZXXX...
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulator' is not defined
Error during measurement: name 'simulato